In [1]:
import torch
import jnu as J
from thesisac.utils import load_run

run_id = "benedict-wilkins/thesis-action-classification/5jjkj851"
trainer, module, data_module, cfg = load_run(run_id)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
import torchvision
data_module.prepare_test_data()
loader = data_module.test_dataloader()
with torch.no_grad():
    for s, a in loader:
        pa = torch.softmax(module(s, a), -1) # if use BCEWithLogitsLoss
        pa = torch.nn.functional.one_hot(pa.argmax(-1), pa.shape[-1]).float()
        ga = torch.nn.functional.one_hot(a[:,0], pa.shape[-1]).float()
        #print(pa.shape, ga.shape)
        err = torch.abs(pa - ga).sum().long().item()
        print(f"errors {err}/{pa.shape[0]} accuracy {1-err/pa.shape[0]}")
        index = torch.abs(pa - ga).sum(-1).bool()
        if index.sum() > 0:
           
            imgs = torch.cat([*s[index].transpose(0,1)], dim=2)
            imgs = torchvision.utils.make_grid(imgs)
            print(imgs.shape)
            print(a[:,0][index])
            
            #J.image(imgs)
        print("-----------------------------------------------------")
